# Reflexivity in a diffusion of innovations model

This code tries to replicate the diffusion of innovations model present in

> *Diffusion dynamics in small-world networks with heterogeneous consumers* from Delre, Sebastiano A., Jager, Wander and Janssen, Marco A., Computational and Mathematical Organization Theory, **13**, 2, 2007.

with some modifications.

It also adds new features that try to introduce reflexivity in it.

## 1. Initialization

### 1.1 External libraries

In [ ]:
from __future__ import division

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
matplotlib.rc('text', usetex=True)

In [ ]:
#plt.style.use('seaborn-paper')
sns.set_style("whitegrid")

### 1.2 Autoreload

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

### 1.3 Internal code

In [ ]:
from algorithm import compute_run, evolution, generate_initial_conditions

In [ ]:
from plots import plot_adopters

## 2. Running the model

### 2.1 Parameters

These are main parameters that control the evolution of the algorithm, and their corresponding variable in the article:

* Network randomness: $r$
* Average number of neighbors: $k$
* Initial proportion (or seed) of adopters: $\delta$
* Coefficient of social influence: $\beta$
* Threshold of adopters among neighbors: $h$
* Quality: $q$
* Total number of consumers: $N$
* Activation sharpness: $\phi$
* Critical mass of adopters: $M_{c}$

----

* Marketing effort: $e_{1}$
* Level: $L$

In [ ]:
parameters = dict(
    randomness = 0.01,
    number_of_neighbors = 30,
    initial_seed = 0.001,
    adopters_threshold = 0.3,
    social_influence = 0.7,
    quality = 0.5,
    number_of_consumers = 1000,
    activation_sharpness = 20,
    critical_mass = 0.85,
    marketing_effort = 0,
    level = 1
    #minimal_utility = 0.5,
)

### 2.2 Simulation

#### 2.2.1 Full run

In [ ]:
data = compute_run(30, parameters, max_time=100)

#### 2.2.2 Single run

In [ ]:
G = generate_initial_conditions(parameters)

In [ ]:
parameters['reflexivity'] = False
set_seed(G, parameters, reset=True)
adopters_no_rx = evolution(G, parameters, max_time=20)

In [ ]:
parameters['reflexivity'] = True
set_seed(G, parameters, reset=True)
adopters_rx = evolution(G, parameters, max_time=20)

## 3. Plots

### 3.1 2d plots

#### 3.1.1 Full run

In [ ]:
plt.figure(figsize=(8, 6))
plot_adopters(data, 'k', parameters['number_of_neighbors'])

In [ ]:
# Plot the cumulative sum of adopters and non-adopters (the usual diffusion curve)
plt.figure(figsize=(8, 6))
plot_adopters(data, 'k', parameters['number_of_neighbors'], cumulative=True)

#### 3.1.2 Single run

In [ ]:
plt.plot(adopters_no_rx, 'b', label='Without Reflexivity')
plt.plot(adopters_rx, 'r', label='With Reflexivity')
plt.legend()
plt.tick_params(axis='both', which='major', labelsize=10)

In [ ]:
# Plot the cumulative sum of adopters and non-adopters (the usual diffusion curve)
plt.figure(figsize=(8, 6))
plt.plot(np.cumsum(adopters_no_rx), '--', color='b', label='Without Reflexivity')
plt.plot(np.cumsum(adopters_rx), color='r', label='With Reflexivity')
plt.xlabel("Time", fontsize=12)
plt.legend(loc='lower right', fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=12)

In [ ]:
plt.figure(figsize=(8, 6))

plt.subplot(2, 1, 1)
#fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(14, 4))
plt.plot(np.cumsum(adopters_no_rx), 'b')
plt.title('Without Reflexivity', fontsize=14)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.subplot(2, 1, 2)
plt.plot(np.cumsum(adopters_rx), 'r')
plt.title('With Reflexivity', fontsize=14)
plt.xlabel("Time", fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()

In [ ]:
# Perform the evolution of a single realization until there is
# complete adoption
print(parameters)
G = generate_initial_conditions(10000, parameters)
indexes = evolution(G, parameters, compute_index=True)

In [ ]:
plt.plot(indexes)

### 3.2 Logistic function form

This explores how the logistic function changes under different values of $k$ and $x_{0}$

In [ ]:
from sympy import plot, symbols, exp

In [ ]:
x = symbols('x')

In [ ]:
def symbolic_logistic(x, k, x0):
    return 1 / ( 1 + exp(-k * (x - x0)) )

In [ ]:
plot(symbolic_logistic(x, 20, 0.85), 1, (x, 0, 1))

In [ ]:
symbolic_logistic(1, 15, 0.8)

### 3.3 Animations

In [ ]:
%matplolib qt

In [ ]:
import matplotlib.animation as animation

In [ ]:
G = generate_initial_conditions(1000, parameters)

In [ ]:
# Plot an animation of the evolution
fig = plt.figure()
positions = nx.spring_layout(G)
animation.FuncAnimation(fig, lambda i: animate(i, G, positions, parameters, test=False),
                        frames=500, interval=3, repeat=False)